<a href="https://colab.research.google.com/github/KevinTheRainmaker/BugBug/blob/main/Send_mail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -q schedule

In [49]:
# import schedule

def sendMail(attachment):
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.base import MIMEBase
    # from string import Template
    from email.mime.text import MIMEText
    from email.mime.image import MIMEImage
    from email import encoders

    #Sender Info
    me = 'leglength28gpa@gmail.com'
    my_password = "don'tgorock"

    # Login
    s = smtplib.SMTP_SSL('smtp.gmail.com')
    s.login(me, my_password)

    # Reciever Info
    email_list = ['gsa15008@gmail.com', 'sweetrainforyou@gmail.com']

    for you in email_list:
        # Email Info
        assert isinstance(template, Template)
        assert isinstance(template_params, dict)
        msg = MIMEMultipart('alternative')
        msg['Subject'] = f""
        msg['From'] = me
        msg['To'] = you

        # Email Contents
        content = template
        part2 = MIMEText(content, 'plain')
        msg.attach(part2)
        part = MIMEBase('application', "octet-stream")
        with open(f"./{attachment}", 'rb') as file:
            part.set_payload(file.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', "attachment", filename=f"{attachment}")
            msg.attach(part)

            # e메일 본문에 이미지를 임베딩한다
            # mime_img = MIMEImage(file.read())
            # msg.attach(mime_img)

        # Sending Email and quit server
        s.sendmail(me, you, msg.as_string())
        print("\nEmail is sented!")

    s.quit()

In [50]:
# Testing
# schedule.every().days.at("23:15").do(sendMail('test.txt'))

In [51]:
# !touch test.txt

In [52]:
sendMail('test.txt')

AttributeError: ignored

In [38]:
import os, copy
import smtplib               # SMTP 라이브러리
from string import Template  # 문자열 템플릿 모듈
from email.mime.multipart import MIMEMultipart
from email.mime.text      import MIMEText
from email.mime.image     import MIMEImage

class EmailHTMLImageContent:
    """e메일에 담길 이미지가 포함된 컨텐츠"""
    def __init__(self, str_subject, str_image_file_name, str_cid_name, template, template_params):
        """이미지파일(str_image_file_name), 컨텐츠ID(str_cid_name)사용된 string template과 딕셔너리형 template_params받아 MIME 메시지를 만든다"""
        assert isinstance(template, Template)
        assert isinstance(template_params, dict)
        self.msg = MIMEMultipart()
        
        # e메일 제목을 설정한다
        self.msg['Subject'] = str_subject # e메일 제목을 설정한다
        
        # e메일 본문을 설정한다
        str_msg  = template.safe_substitute(**template_params) # ${변수} 치환하며 문자열 만든다
        mime_msg = MIMEText(str_msg, 'html')                   # MIME HTML 문자열을 만든다
        self.msg.attach(mime_msg)
        
        # e메일 본문에 이미지를 임베딩한다
        assert template.template.find("cid:" + str_cid_name) >= 0, 'template must have cid for embedded image.'
        assert os.path.isfile(str_image_file_name), 'image file does not exist.'        
        with open(str_image_file_name, 'rb') as img_file:
            mime_img = MIMEImage(img_file.read())
            mime_img.add_header('Content-ID', '<' + str_cid_name + '>')
        self.msg.attach(mime_img)
        
    def get_message(self, str_from_email_addr, str_to_eamil_addrs):
        """발신자, 수신자리스트를 이용하여 보낼메시지를 만든다 """
        mm = copy.deepcopy(self.msg)
        mm['From'] = str_from_email_addr          # 발신자 
        mm['To']   = ",".join(str_to_eamil_addrs) # 수신자리스트 
        return mm

class emailSender:
    """e메일 발송자"""
    #Sender Info
    me = 'leglength28gpa@gmail.com'
    my_password = "don'tgorock"

    def __init__(self, str_host, me, my_password, num_port=25):
        """호스트와 포트번호로 SMTP로 연결한다 """
        self.str_host = str_host
        self.num_port = num_port
        self.ss = smtplib.SMTP_SSL('smtp.gmail.com')
        self.me = me
        self.my_password = my_password
        # SMTP인증이 필요하면 아래 주석을 해제하세요.
        # self.ss.starttls() # TLS(Transport Layer Security) 시작
        # self.ss.login('leglength28gpa@gmail.com', "don'tgorock") # 메일서버에 연결한 계정과 비밀번호
    
    def send_message(self, emailContent, str_from_email_addr, str_to_email_addrs):
        """e메일을 발송한다 """
        self.ss.login(self.me, self.my_password)
        cc = emailContent.get_message(str_from_email_addr, str_to_email_addrs)
        self.ss.send_message(cc, from_addr=str_from_email_addr, to_addrs=str_to_email_addrs)
        del cc

In [39]:
str_subject = 'hello with image'
template = Template("""<html>
                            <head></head>
                            <body>
                                Hi ${NAME}.<br>
                                <img src="cid:my_image1"><br>
                                This is a test message.
                            </body>
                        </html>""")
template_params       = {'NAME':'Environment Lover'}
str_image_file_name   = 'justice.png'
str_cid_name          = 'my_image1'
emailHTMLImageContent = EmailHTMLImageContent(str_subject, str_image_file_name, str_cid_name, template, template_params)

str_from_email_addr = 'leglength28gpa@gmail.com' # 발신자
str_to_email_addrs  = ['sweetrainforyou@gmail.com', 'gsa15008@gmail.com'] # 수신자리스트
emailSender.send_message(emailHTMLImageContent, str_from_email_addr, str_to_email_addrs)

TypeError: ignored

In [10]:
import smtplib
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email import encoders
import os


send_Mail = 'gsa15008@gmail.com'
smtp = smtplib.SMTP_SSL('smtp.gmail.com',465)
smtp.login('leglength28gpa@gmail.com', "don'tgorock")                        #보낼 이메일의 이메일,비번


msg = MIMEMultipart('alternative')
cont = MIMEText('바다거북 이야기', 'plain', 'utf-8')
msg['Subject'] = 'The 1st News Letter'
msg['From'] = 'leglength28gpa@gmail.com'                             #보낼 사람의 이메일
msg['To'] = send_Mail
msg.attach(cont)
path = r'/content/justice.png' #보낼 이미지의 경로
part = MIMEBase("application", "octet-stream")
part.set_payload(open(path, 'rb').read())
encoders.encode_base64(part)
part.add_header('Content-Disposition',
        'attachment; filename="%s"'% os.path.basename(path))
mime_img = MIMEImage(part.read())
msg.attach(part)
smtp.sendmail('leglength28gpa@gmail.com', send_Mail, msg.as_string())   #보낼 사람의 이메일
smtp.quit()

(221, b'2.0.0 closing connection hk18sm773676pjb.41 - gsmtp')

In [11]:
import smtplib
from email.message import EmailMessage
import imghdr

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465

message = EmailMessage()
message.set_content("Fuck You Hackerton")

message["Subject"] = "제ㅔㅔㅔㅔㅔㅔㅔㅔㅔ목"
message["From"] = "leglength28gpa@gmail.com"
message["To"] = "sweetrainforyou@gmail.com"

with open("justice.png","rb") as image:
    image_file = image.read()

image_type = imghdr.what('dgdr',image_file)
message.add_attachment(image_file,maintype='image',subtype=image_type)

smtp = smtplib.SMTP_SSL(SMTP_SERVER,SMTP_PORT)
smtp.login("leglength28gpa@gmail.com","don'tgorock")
smtp.send_message(message)
smtp.quit()

(221, b'2.0.0 closing connection pc4sm3296519pjb.3 - gsmtp')

In [20]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

sender = "leglength28gpa@gmail.com"
recipient = 'kgbko1117@naver.com'
cc = "sweetrainforyou@gmail.com"

title = '쉬이이ㅣ이발'
contents = '''
뻑유 썬더
'''

attach_file_name_1 = 'justice.png'
attach_file_name_2 = 'justice2.png'

msg = MIMEMultipart()
msg['From'] = sender
msg['To'] = recipient
msg['Subject'] = title
msg['Cc'] = cc


msg.attach(MIMEText(contents, 'plain'))

##### attach file 1
attached_file = open(attach_file_name_1, 'rb').read()
att = MIMEBase('application', 'vnd.openxmlformats-officedocument.spreadsheetml.sheet')

att.set_payload(attached_file)
encoders.encode_base64(att)
att.add_header('Content-Disposition', 'attachment', filename=attach_file_name_1)

msg.attach(att)



##### attach file 2
# msg.attach(MIMEText(contents, 'plain'))
attached_file = open(attach_file_name_2, 'rb').read()
att = MIMEBase('application', 'vnd.openxmlformats-officedocument.spreadsheetml.sheet')

att.set_payload(attached_file)
encoders.encode_base64(att)
att.add_header('Content-Disposition', 'attachment', filename=attach_file_name_2)

msg.attach(att)



smtp_server = smtplib.SMTP_SSL('smtp.gmail.com',465)
smtp_server.login('leglength28gpa@gmail.com', "don'tgorock")                        #보낼 이메일의 이메일,비번
smtp_server.sendmail(from_addr=sender, to_addrs=recipient, msg=msg.as_string())
smtp_server.quit()

(221, b'2.0.0 closing connection d12sm2370790pgk.29 - gsmtp')